In [1]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTENC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
import pickle
import optuna
from xgboost import XGBClassifier

# Register API for Financial Modeling Prep (Financial Statements and Company Fundamentals)
# https://site.financialmodelingprep.com/developer/
# Register API for Federal Reserve Economic Data (For Macroeconomics Data)
# https://fred.stlouisfed.org/docs/api/fred/
# Yahoo Finance does not need an API

warnings.filterwarnings('ignore')

In [3]:
X_test_transformed = pd.read_csv('X_test_transformed.csv')
y_test = pd.read_csv('y_test.csv')
X_train_oversampled = pd.read_csv('X_train_oversampled.csv')
y_train_oversampled = pd.read_csv('y_train_oversampled.csv')
X_validate_transformed = pd.read_csv('X_validate_transformed.csv')
y_validate = pd.read_csv('y_validate.csv')

In [4]:
# testing if file read correctly
X_train_oversampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7236 entries, 0 to 7235
Data columns (total 92 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   industry                                              7236 non-null   float64
 1   sector                                                7236 non-null   float64
 2   symbol                                                7236 non-null   float64
 3   year                                                  7236 non-null   float64
 4   adjDividend                                           7236 non-null   float64
 5   dps_growth                                            7236 non-null   float64
 6   dps_growth_rate                                       7236 non-null   float64
 7   interestRate                                          7236 non-null   float64
 8   interestRate_percentage_change                        7236

In [5]:
# using only top 8 features
cols = ['dps_growth_rate','dps_growth','interestRate','year','dividendYield','interestRate_percentage_change','freeCashFlowPerShare','adjDividend']
X_train_oversampled = X_train_oversampled[cols]
X_validate_transformed = X_validate_transformed[cols]
X_test_transformed = X_test_transformed[cols]
X_test_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dps_growth_rate                 479 non-null    float64
 1   dps_growth                      479 non-null    float64
 2   interestRate                    479 non-null    float64
 3   year                            479 non-null    float64
 4   dividendYield                   479 non-null    float64
 5   interestRate_percentage_change  479 non-null    float64
 6   freeCashFlowPerShare            479 non-null    float64
 7   adjDividend                     479 non-null    float64
dtypes: float64(8)
memory usage: 30.1 KB


In [6]:
# Logistic Regression
def objective_function(trial):
    C = trial.suggest_float('C', 0.1, 10, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver='liblinear',
        n_jobs=-1
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, C: {C}, penalty: {penalty}, ROC-AUC: {roc_auc}")
    return roc_auc


study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_function, n_trials=100)

best_params_lr = study_lr.best_params
print("Best Parameters: ", best_params_lr)
print("Best ROC-AUC Score: ", study_lr.best_value)

# Create and save model
best_model_lr = LogisticRegression(**best_params_lr, solver='liblinear', n_jobs=-1)
with open('best_models_lr_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_lr, file)

[I 2024-05-13 19:37:30,581] A new study created in memory with name: no-name-8399fe2c-8fce-48b4-ad07-e10291406f63
[I 2024-05-13 19:37:31,268] Trial 0 finished with value: 0.7162263290996873 and parameters: {'C': 0.586202521120192, 'penalty': 'l2'}. Best is trial 0 with value: 0.7162263290996873.


Trial 0, C: 0.586202521120192, penalty: l2, ROC-AUC: 0.7162263290996873


[I 2024-05-13 19:37:32,242] Trial 1 finished with value: 0.7463089245456522 and parameters: {'C': 3.9640007504045256, 'penalty': 'l1'}. Best is trial 1 with value: 0.7463089245456522.


Trial 1, C: 3.9640007504045256, penalty: l1, ROC-AUC: 0.7463089245456522


[I 2024-05-13 19:37:33,141] Trial 2 finished with value: 0.7312900209248396 and parameters: {'C': 0.18182473561720253, 'penalty': 'l1'}. Best is trial 1 with value: 0.7463089245456522.


...

Trial 97, C: 8.042821526159976, penalty: l2, ROC-AUC: 0.7414967892912174


[I 2024-05-13 19:40:30,602] Trial 98 finished with value: 0.746653450006318 and parameters: {'C': 9.984210741535154, 'penalty': 'l1'}. Best is trial 71 with value: 0.7466820964297234.


Trial 98, C: 9.984210741535154, penalty: l1, ROC-AUC: 0.746653450006318


[I 2024-05-13 19:40:31,927] Trial 99 finished with value: 0.7449632798891406 and parameters: {'C': 1.255055301594677, 'penalty': 'l1'}. Best is trial 71 with value: 0.7466820964297234.


Trial 99, C: 1.255055301594677, penalty: l1, ROC-AUC: 0.7449632798891406
Best Parameters:  {'C': 9.920488934355904, 'penalty': 'l1'}
Best ROC-AUC Score:  0.7466820964297234


In [7]:
# Decision Tree
def objective_function(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, max_depth: {max_depth}, min_samples_split: {min_samples_split}, "
          f"min_samples_leaf: {min_samples_leaf}, criterion: {criterion}, ROC-AUC: {roc_auc}")
    return roc_auc

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_function, n_trials=100)

best_params_dt = study_dt.best_params
print("Best Parameters: ", best_params_dt)
print("Best ROC-AUC Score: ", study_dt.best_value)

# Create and save model
best_model_dt = DecisionTreeClassifier(**best_params_dt)
with open('best_models_dt_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_dt, file)

[I 2024-05-13 19:40:31,957] A new study created in memory with name: no-name-104b400f-dec2-4465-b58e-e0e5a7430c38
[I 2024-05-13 19:40:32,286] Trial 0 finished with value: 0.9605684227332679 and parameters: {'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 4, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9605684227332679.


Trial 0, max_depth: 10, min_samples_split: 6, min_samples_leaf: 4, criterion: entropy, ROC-AUC: 0.9605684227332679


[I 2024-05-13 19:40:32,508] Trial 1 finished with value: 0.9273296228082654 and parameters: {'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'gini'}. Best is trial 0 with value: 0.9605684227332679.


Trial 1, max_depth: 6, min_samples_split: 2, min_samples_leaf: 5, criterion: gini, ROC-AUC: 0.9273296228082654


[I 2024-05-13 19:40:32,800] Trial 2 finished with value: 0.9571814288364411 and parameters: {'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9605684227332679.


...

Trial 97, max_depth: 10, min_samples_split: 9, min_samples_leaf: 10, criterion: gini, ROC-AUC: 0.9643080741983002


[I 2024-05-13 19:41:02,911] Trial 98 finished with value: 0.9631035372680881 and parameters: {'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 82 with value: 0.9647926389890351.


Trial 98, max_depth: 10, min_samples_split: 9, min_samples_leaf: 10, criterion: gini, ROC-AUC: 0.9631035372680881


[I 2024-05-13 19:41:03,353] Trial 99 finished with value: 0.9625915868089958 and parameters: {'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 9, 'criterion': 'gini'}. Best is trial 82 with value: 0.9647926389890351.


Trial 99, max_depth: 10, min_samples_split: 8, min_samples_leaf: 9, criterion: gini, ROC-AUC: 0.9625915868089958
Best Parameters:  {'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 10, 'criterion': 'gini'}
Best ROC-AUC Score:  0.9647926389890351


In [8]:
# KNN
def objective_function(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 10)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        p=p,
        metric=metric
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_neighbors: {n_neighbors}, weights: {weights}, p: {p}, metric: {metric}, "
          f"ROC-AUC: {roc_auc}")
    return roc_auc


study_knn = optuna.create_study(direction="maximize")
study_knn.optimize(objective_function, n_trials=100)

best_params_knn = study_knn.best_params
print("Best Parameters: ", best_params_knn)
print("Best ROC-AUC Score: ", study_knn.best_value)

# Create and save model
best_model_knn = KNeighborsClassifier(**best_params_knn)
with open('best_models_knn_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_knn, file)

[I 2024-05-13 19:41:03,384] A new study created in memory with name: no-name-76c92874-4e9b-4351-be1c-99e7b8d00630
[I 2024-05-13 19:41:03,739] Trial 0 finished with value: 0.924409583503105 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 2, 'metric': 'euclidean'}. Best is trial 0 with value: 0.924409583503105.


Trial 0, n_neighbors: 7, weights: uniform, p: 2, metric: euclidean, ROC-AUC: 0.924409583503105


[I 2024-05-13 19:41:04,099] Trial 1 finished with value: 0.9230246250844848 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 3, 'metric': 'euclidean'}. Best is trial 0 with value: 0.924409583503105.


Trial 1, n_neighbors: 9, weights: uniform, p: 3, metric: euclidean, ROC-AUC: 0.9230246250844848


[I 2024-05-13 19:41:04,713] Trial 2 finished with value: 0.9224677088387698 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'p': 4, 'metric': 'minkowski'}. Best is trial 0 with value: 0.924409583503105.


...

Trial 97, n_neighbors: 10, weights: distance, p: 4, metric: manhattan, ROC-AUC: 0.9482379985090897


[I 2024-05-13 19:41:34,476] Trial 98 finished with value: 0.934080327532868 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 3, 'metric': 'euclidean'}. Best is trial 20 with value: 0.9482379985090897.


Trial 98, n_neighbors: 9, weights: distance, p: 3, metric: euclidean, ROC-AUC: 0.934080327532868


[I 2024-05-13 19:41:34,736] Trial 99 finished with value: 0.9444288569594713 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'p': 4, 'metric': 'manhattan'}. Best is trial 20 with value: 0.9482379985090897.


Trial 99, n_neighbors: 5, weights: distance, p: 4, metric: manhattan, ROC-AUC: 0.9444288569594713
Best Parameters:  {'n_neighbors': 10, 'weights': 'distance', 'p': 4, 'metric': 'manhattan'}
Best ROC-AUC Score:  0.9482379985090897


In [9]:
# Random Forest
def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, "
          f"min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}, ROC-AUC: {roc_auc}")
    return roc_auc


study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_function, n_trials=100)

best_params_rf = study_rf.best_params
print("Best Parameters: ", best_params_rf)
print("Best ROC-AUC: Score: ", study_rf.best_value)

# Create and save model
best_model_rf = RandomForestClassifier(**best_params_rf, n_jobs=-1)
with open('best_models_rf_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_rf, file)

[I 2024-05-13 19:41:34,760] A new study created in memory with name: no-name-2241285a-1c5d-4510-89c7-1adea6620cca
[I 2024-05-13 19:41:35,426] Trial 0 finished with value: 0.8914686969878625 and parameters: {'n_estimators': 25, 'max_depth': 2, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8914686969878625.


Trial 0, n_estimators: 25, max_depth: 2, min_samples_split: 2, min_samples_leaf: 5, ROC-AUC: 0.8914686969878625


[I 2024-05-13 19:41:36,303] Trial 1 finished with value: 0.9715798298802893 and parameters: {'n_estimators': 26, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9715798298802893.


Trial 1, n_estimators: 26, max_depth: 6, min_samples_split: 2, min_samples_leaf: 5, ROC-AUC: 0.9715798298802893


[I 2024-05-13 19:41:37,131] Trial 2 finished with value: 0.9815596763229631 and parameters: {'n_estimators': 21, 'max_depth': 8, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is trial 2 with value: 0.9815596763229631.


...

Trial 97, n_estimators: 45, max_depth: 9, min_samples_split: 5, min_samples_leaf: 1, ROC-AUC: 0.9862181229511542


[I 2024-05-13 19:43:56,850] Trial 98 finished with value: 0.988601485905107 and parameters: {'n_estimators': 47, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 44 with value: 0.9887328165789173.


Trial 98, n_estimators: 47, max_depth: 10, min_samples_split: 5, min_samples_leaf: 1, ROC-AUC: 0.988601485905107


[I 2024-05-13 19:43:59,243] Trial 99 finished with value: 0.988609728308923 and parameters: {'n_estimators': 47, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 44 with value: 0.9887328165789173.


Trial 99, n_estimators: 47, max_depth: 10, min_samples_split: 6, min_samples_leaf: 2, ROC-AUC: 0.988609728308923
Best Parameters:  {'n_estimators': 39, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 1}
Best ROC-AUC: Score:  0.9887328165789173


In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train_oversampled)
X_test = sc.transform(X_test_transformed)

# Support Vector Machine
def objective_function(trial):
    C = trial.suggest_float('C', 0.01, 10, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['auto', 'scale'])

    model = SVC(
        C=C,
        kernel=kernel,
        gamma=gamma,
        probability=True
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, C: {C}, kernel: {kernel}, gamma: {gamma}, ROC-AUC: {roc_auc}")
    return roc_auc

study_svm = optuna.create_study(direction="maximize")
study_svm.optimize(objective_function, n_trials=100)

best_params_svm = study_svm.best_params
print("Best Parameters: ", best_params_svm)
print("Best ROC-AUC: Score: ", study_svm.best_value)

# Create and save model
best_model_svm = SVC(**best_params_svm, probability=True)
with open('best_models_svm_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_svm, file)

[I 2024-05-13 19:43:59,272] A new study created in memory with name: no-name-189481d1-79b2-4bf8-abed-9507f600fa3b
[I 2024-05-13 19:44:39,453] Trial 0 finished with value: 0.7616713847086425 and parameters: {'C': 0.4393578465731463, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.7616713847086425.


Trial 0, C: 0.4393578465731463, kernel: linear, gamma: auto, ROC-AUC: 0.7616713847086425


[I 2024-05-13 19:45:41,252] Trial 1 finished with value: 0.7618333133639021 and parameters: {'C': 2.609897267396315, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 1 with value: 0.7618333133639021.


Trial 1, C: 2.609897267396315, kernel: linear, gamma: auto, ROC-AUC: 0.7618333133639021


[I 2024-05-13 19:46:45,446] Trial 2 finished with value: 0.718671779094353 and parameters: {'C': 0.014399085782224975, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 1 with value: 0.7618333133639021.


...

Trial 74, C: 5.927598287968757, kernel: rbf, gamma: auto, ROC-AUC: 0.9569683118902595


[I 2024-05-13 20:29:30,893] Trial 75 finished with value: 0.7618493559309611 and parameters: {'C': 9.989976821406506, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 46 with value: 0.9614003128741233.


Trial 75, C: 9.989976821406506, kernel: linear, gamma: auto, ROC-AUC: 0.7618493559309611


[I 2024-05-13 20:29:55,356] Trial 76 finished with value: 0.9594507158872194 and parameters: {'C': 8.018208266519167, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 46 with value: 0.9614003128741233.


Trial 76, C: 8.018208266519167, kernel: rbf, gamma: auto, ROC-AUC: 0.9594507158872194


In [ ]:
label_encoder = LabelEncoder()
# Fit the encoder and transform the target variable
y_train_oversampled_encoded = label_encoder.fit_transform(y_train_oversampled)
# This suppresses printing logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.9, log=True)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        use_label_encoder=False,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled_encoded, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, learning_rate: {learning_rate},"
          f"min_child_weight: {min_child_weight}, subsample: {subsample}, colsample_bytree: {colsample_bytree}, "
          f"gamma: {gamma}, reg_alpha: {reg_alpha}, reg_lambda: {reg_lambda}, ROC-AUC: {roc_auc}")
    return roc_auc


study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_function, n_trials=100)

best_params_xgb = study_xgb.best_params
print("Best Parameters: ", best_params_xgb)
print("Best ROC-AUC Score: ", study_xgb.best_value)

best_model_xgb = XGBClassifier(**best_params_xgb, use_label_encoder=False, n_jobs=-1)
with open('best_models_xgb_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_xgb, file)

In [ ]:
with open('best_models_lr_8_features.pkl', 'rb') as file:
    best_model_lr = pickle.load(file)
with open('best_models_dt_8_features.pkl', 'rb') as file:
    best_model_dt = pickle.load(file)
with open('best_models_knn_8_features.pkl', 'rb') as file:
    best_model_knn = pickle.load(file)
with open('best_models_rf_8_features.pkl', 'rb') as file:
    best_model_rf = pickle.load(file)
with open('best_models_xgb_8_features.pkl', 'rb') as file:
    best_model_xgb = pickle.load(file)
with open('best_models_svm_8_features.pkl', 'rb') as file:
    best_model_svm = pickle.load(file)

print("Testing Performances...Please wait")
best_model_lr.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_lr.predict_proba(X_test_transformed)[:, 1]
lr_performance = roc_auc_score(y_test, predicted_probs)

best_model_dt.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_dt.predict_proba(X_test_transformed)[:, 1]
dt_performance = roc_auc_score(y_test, predicted_probs)

best_model_knn.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_knn.predict_proba(X_test_transformed)[:, 1]
knn_performance = roc_auc_score(y_test, predicted_probs)

best_model_rf.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_rf.predict_proba(X_test_transformed)[:, 1]
rf_performance = roc_auc_score(y_test, predicted_probs)

best_model_xgb.fit(X_train_oversampled, y_train_oversampled_encoded)
predicted_probs = best_model_xgb.predict_proba(X_test_transformed)[:, 1]
xgb_performance = roc_auc_score(y_test, predicted_probs)

best_model_svm.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_svm.predict_proba(X_test)[:, 1]
svm_performance = roc_auc_score(y_test, predicted_probs)

# Test performance of the models are
print(f"Logistic Regression Test ROCAUC: {lr_performance}")
print(f"Decision Tree Test ROCAUC: {dt_performance}")
print(f"KNN Test ROCAUC: {knn_performance}")
print(f"Random Forest Test ROCAUC: {rf_performance}")
print(f"XGBoost Test ROCAUC: {xgb_performance}")
print(f"SVM Test ROCAUC: {svm_performance}")